In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Step 1: Download stock data from Yahoo Finance
def download_stock_data(ticker, period='5y'):
    stock_data = yf.download(ticker, period=period)
    return stock_data

# Step 2: Preprocess the data
def preprocess_data(data, feature_col='Close', seq_length=60):
    # Use 'Close' prices to predict trends
    data = data[[feature_col]]

    # Normalize the data using MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    # Create sequences of data points for LSTM input
    X, y = [], []
    for i in range(seq_length, len(scaled_data)):
        X.append(scaled_data[i-seq_length:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)

    # Reshape the data to be compatible with LSTM (samples, timesteps, features)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    return X, y, scaler

# Step 3: Build the LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=25))
    model.add(Dense(units=1))  # Predicting a single output value (next price)

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Step 4: Train the LSTM model
def train_lstm_model(model, X_train, y_train, epochs=10, batch_size=64):
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)
    return model

# Step 5: Make predictions and evaluate
def predict_and_evaluate(model, X_test, y_test, scaler):
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)  # Rescale back to original values
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Calculate the Root Mean Squared Error (RMSE)
    rmse = np.sqrt(np.mean((predictions - y_test) ** 2))
    return predictions, rmse

# Step 6: Trading simulation logic
def simulate_trading(predictions, actual_prices, initial_balance=10000, shares=0):
    balance = initial_balance
    total_shares = shares
    trade_log = []

    for i in range(1, len(predictions)):
        predicted_price = predictions[i]
        actual_price = actual_prices[i]

        # Buy logic: if the predicted price is higher than the current price, buy shares
        if predicted_price > actual_prices[i-1] and balance > actual_price:
            shares_to_buy = balance // actual_price
            balance -= shares_to_buy * actual_price
            total_shares += shares_to_buy
            trade_log.append(f"Bought {shares_to_buy} shares at {actual_price}, Balance: {balance}, Shares: {total_shares}")

        # Sell logic: if the predicted price is lower than the current price, sell shares
        elif predicted_price < actual_prices[i-1] and total_shares > 0:
            balance += total_shares * actual_price
            trade_log.append(f"Sold {total_shares} shares at {actual_price}, Balance: {balance}")
            total_shares = 0

    # Final balance after selling any remaining shares
    if total_shares > 0:
        balance += total_shares * actual_prices[-1]
        trade_log.append(f"Final Sale of {total_shares} shares at {actual_prices[-1]}, Final Balance: {balance}")

    profit_loss = balance - initial_balance
    return trade_log, profit_loss

In [ ]:
# Step 6: Trading simulation logic
def simulate_trading(predictions, actual_prices, dates, initial_balance=10000, shares=0):
    balance = initial_balance
    total_shares = shares
    trade_log = []

    for i in range(1, len(predictions)):
        predicted_price = predictions[i]
        actual_price = actual_prices[i]
        date = dates[i]

        # Buy logic: if the predicted price is higher than the current price, buy shares
        if predicted_price > actual_prices[i-1] and balance > actual_price:
            shares_to_buy = balance // actual_price
            balance -= shares_to_buy * actual_price
            total_shares += shares_to_buy
            trade_log.append(f"Bought {shares_to_buy} shares at {actual_price} on {date}, Balance: {balance}, Shares: {total_shares}")

        # Sell logic: if the predicted price is lower than the current price, sell shares
        elif predicted_price < actual_prices[i-1] and total_shares > 0:
            balance += total_shares * actual_price
            trade_log.append(f"Sold {total_shares} shares at {actual_price} on {date}, Balance: {balance}")
            total_shares = 0

    # Final balance after selling any remaining shares
    if total_shares > 0:
        balance += total_shares * actual_prices[-1]
        trade_log.append(f"Final Sale of {total_shares} shares at {actual_prices[-1]} on {dates[-1]}, Final Balance: {balance}")

    profit_loss = balance - initial_balance
    return trade_log, profit_loss

# Step 7: Run the entire pipeline with trading simulation
def run_stock_prediction_with_simulation(ticker, period='5y', seq_length=60):
    # Download and preprocess the data
    stock_data = download_stock_data(ticker, period)
    X, y, scaler = preprocess_data(stock_data, seq_length=seq_length)

    # Get the corresponding dates for the test set
    dates = stock_data.index[seq_length:]  # Dates aligned with the sequences

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Adjust the corresponding dates for the test set
    test_dates = dates[-len(X_test):]

    # Create and train the LSTM model
    model = create_lstm_model(input_shape=(X_train.shape[1], 1))
    model = train_lstm_model(model, X_train, y_train, epochs=10)

    # Make predictions and evaluate the model
    predictions, rmse = predict_and_evaluate(model, X_test, y_test, scaler)

    # Rescale the actual test prices
    actual_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Simulate trading based on model predictions and include dates
    trade_log, profit_loss = simulate_trading(predictions.flatten(), actual_prices.flatten(), test_dates)

    print(f"Root Mean Squared Error: {rmse}")
    print(f"Final Profit/Loss: {profit_loss}")
    for log in trade_log:
        print(log)

    return stock_data, predictions, trade_log, profit_loss

# Example usage:
if __name__ == "__main__":
    ticker_symbol = 'AAPL'  # Example stock ticker (Apple Inc.)
    stock_data, predictions, trade_log, profit_loss = run_stock_prediction_with_simulation(ticker_symbol)


[*********************100%***********************]  1 of 1 completed

Epoch 1/10



/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - loss: 0.1020
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - loss: 0.0115
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - loss: 0.0061
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 0.0046
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0033
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 0.0033
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.0037
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 0.0028
Epoch 9/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - loss: 0.0027
Epoch 10/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 0.0029
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step
Root Mean Squared Error: 7.33112419508141
Final Profit/Loss: 2696.751602172855
Bought 58.0 shares at 171.10000610351562 on 2023-10-25 00:00:00, Balance: 76.19964599609375, Shares: 58.0
Sold 58.0 shares at 177.57000732421875 on 2023-11-02 00:00:00, Balance: 10375.260070800781
Bought 53.0 shares at 193.0500030517

In [ ]:
# Step 6: Trading strategy based on LSTM predictions
def trading_strategy(predictions, real_prices):
    buy, sell, Close_profit, Stop_loss = [], [], [], []
    Close_order_buy, Close_order_sell = [], []
    position_Buy, position_Sell = None, None
    profit_threshold, loss_threshold = 0.4, 0.4  # Profit and loss thresholds

    for i in range(len(predictions)):
        pred = predictions[i]
        real = real_prices[i]

        # Ensure that all arrays are updated in every iteration
        if position_Buy != "Buy" and position_Sell != "Sell" and pred > real:
            # Buy condition
            buy.append(real)
            sell.append(np.nan)
            Close_profit.append(np.nan)
            Stop_loss.append(np.nan)
            Close_order_buy.append(real)
            position_Buy = "Buy"

        elif position_Sell != "Sell" and position_Buy != "Buy" and pred < real:
            # Sell condition
            sell.append(real)
            buy.append(np.nan)
            Close_profit.append(np.nan)
            Stop_loss.append(np.nan)
            Close_order_sell.append(real)
            position_Sell = "Sell"

        elif position_Sell == "Sell":
            # Closing sell position
            if Close_order_sell and Close_order_sell[-1] + loss_threshold <= real:
                Stop_loss.append(real)  # Take profit
                Close_profit.append(np.nan)
                position_Sell = None
            elif Close_order_sell and Close_order_sell[-1] - profit_threshold >= real:
                Close_profit.append(real)  # Stop loss
                Stop_loss.append(np.nan)
                position_Sell = None
            else:
                Close_profit.append(np.nan)
                Stop_loss.append(np.nan)

        elif position_Buy == "Buy":
            # Closing buy position
            if Close_order_buy and Close_order_buy[-1] + profit_threshold <= real:
                Close_profit.append(real)  # Take profit
                Stop_loss.append(np.nan)
                position_Buy = None
            elif Close_order_buy and Close_order_buy[-1] - loss_threshold >= real:
                Stop_loss.append(real)  # Stop loss
                Close_profit.append(np.nan)
                position_Buy = None
            else:
                Close_profit.append(np.nan)
                Stop_loss.append(np.nan)

        else:
            # Append np.nan to keep array lengths consistent
            buy.append(np.nan)
            sell.append(np.nan)
            Close_profit.append(np.nan)
            Stop_loss.append(np.nan)

    # Add to DataFrame for analysis, ensuring all lists are the same length as `predictions`
    result_df = pd.DataFrame({
        'Predictions': predictions,
        'Real Prices': real_prices,
        'Buy': buy[:len(predictions)],
        'Sell': sell[:len(predictions)],
        'Close Profit': Close_profit[:len(predictions)],
        'Stop Loss': Stop_loss[:len(predictions)]
    })

    return result_df

In [ ]:


# Step 7: Run the entire pipeline
def run_stock_prediction(ticker, period='5y', seq_length=60):
    # Download and preprocess the data
    stock_data = download_stock_data(ticker, period)
    X, y, scaler = preprocess_data(stock_data, seq_length=seq_length)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Create and train the LSTM model
    model = create_lstm_model(input_shape=(X_train.shape[1], 1))
    model = train_lstm_model(model, X_train, y_train, epochs=10)

    # Make predictions and evaluate the model
    predictions, rmse = predict_and_evaluate(model, X_test, y_test, scaler)

    # Use the trading strategy with the predictions and real prices
    real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))
    result_df = trading_strategy(predictions.flatten(), real_prices.flatten())

    print(f"Root Mean Squared Error: {rmse}")
    print(result_df.tail())  # Show the last few trades
    return stock_data, predictions, result_df

# Example usage:
if __name__ == "__main__":
    ticker_symbol = 'AAPL'  # Example stock ticker (Apple Inc.)
    stock_data, predictions, result_df = run_stock_prediction(ticker_symbol)


[*********************100%***********************]  1 of 1 completed

Epoch 1/10



/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - loss: 0.0844
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 101ms/step - loss: 0.0109
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - loss: 0.0051
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: 0.0038
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 0.0032
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 0.0029
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0028
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0029
Epoch 9/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 0.0028
Epoch 10/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0025
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step


ValueError: All arrays must be of the same length